# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [11]:
# Get the datasets
!curl --output train.dat http://huang.eng.unt.edu/CSCE-5218/train.dat
!curl --output test.dat http://huang.eng.unt.edu/CSCE-5218/test.dat


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11244  100 11244    0     0  88068      0 --:--:-- --:--:-- --:--:-- 87843
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2844  100  2844    0     0  42615      0 --:--:-- --:--:-- --:--:-- 43090


In [12]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0


A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [13]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    #TODO: Return dot product of array 1 and array 2
    return sum(a*b for a, b in zip(array1, array2))


def sigmoid(x):
    #TODO: Return outpout of sigmoid function on x
    return 1 / (1 + math.exp(-x))

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    #TODO: return the output of the model 
    return sigmoid(dot_product(weight, instance[:-1]))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    #TODO: return the prediction of the model
    return 1 if output(weights, instance) >= 0.5 else 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    #TODO: name this step
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            #TODO: name these steps
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            #TODO: name these steps
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [14]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)

The reason the first code snippet is used is because the first snippet calculates the continuous output of the perceptron model using the sigmoid function, which assists in computing the gradient and subsequent updates of weights. By using the sigmoid function, a smooth gradient is used to adjust the weights in a far more precise manner based on how far off the prediction is. The 'predict' function only provides a binary 1 or 0 depending on whether the sigmoid function is above or below 0.5. By using a binary output, gradient information, which is necessary or weight updates during training.The gradient allows for nuanced adjustments to the weights that the binary model would not be able to provide. The prediction would be either 0 or 1, not a continuous value that represents the confidence of the prediction.

### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 


In [15]:
# Load the datasets
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")

# Define the arrays for the hyperparameters
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]         # learning rate

# Iterate over each combination of hyperparameters
for lr in lr_array:
    for percent in tr_percent:
        size = round(len(instances_tr) * percent / 100)
        training_subset = instances_tr[:size]
        
        for epochs in num_epochs:
            # Train the perceptron with the current subset, learning rate, and epochs
            weights = train_perceptron(training_subset, lr, epochs)
            
            # Calculate the accuracy on the test dataset
            accuracy = get_accuracy(weights, instances_te)
            
            # Print the results for this combination
            print(f"#tr: {len(training_subset)}, epochs: {epochs}, learning rate: {lr:.3f}; "
                  f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}%")


#tr: 20, epochs: 5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 20, epochs: 10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 20, epochs: 20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 20, epochs: 50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 40, epochs: 5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 40, epochs: 10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 40, epochs: 20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 40, epochs: 50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 100, epochs: 5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 100, epochs: 10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0%
#tr: 100, epochs: 20, learn

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)

A. The data shows that using the entire training dataset with a higher learning rate (0.05) and more epochs leads to the highest accuracy of 80%. For this problem, it is clear that having more data available for training helps the model to learn and generalize more effectively when using new data. The improvement in accuracy is not linear with the amount of training data, and there are diminishing returns as the dataset size increases, which indicates that while using the entire dataset can be beneficial, it is not always necessary to achieve high accuracy.

B. When comparing the two runs, the difference in accuracy can be attributed to the learning rate. A higher learning rate (0.05) with fewer data points (100 instances) resulted in better accuracy than a lower learning rate (0.005) with more data (200 instances). A suitable learning rate can help the model to converge more quickly to a good solution, even with less data, which depicts the importance of the learning rate in the training process. However, it also brings to attention the trade-off and balance required between learning rate and dataset size, meaning that a higher learning rate may not always be better, especially if it causes the model to overshoot the minimum of the loss function.

C. The highest accuracy observed is 80.0% with specific hyperparameters as presented . Achieving higher accuracy (above 80.0%) could be possible by further tuning hyperparameters, introducing regularization to prevent overfitting, or using more sophisticated models like deep neural networks that can capture complex patterns in the data more effectively. However, with the simple perceptron model in this problem, there might be a limit to how much accuracy can be improved without overfitting the training data.

D. Increasing the number of epochs in model training, with all other hyperparameters held constant, does not guarantee improved performance indefinitely because it can lead to overfitting. When overfitting occurs, the model learns the noise in the training data rather than the underlying pattern. This overfitting results in poorer generalization to new data. The optimal number of epochs depends on various factors, including the complexity of the model along with the learning rate and the size and diversity of training data. Monitoring the model's performance on a validation set to determine the best point to stop training is essential.
